In [1]:
%cd ../../spell_correction

/Users/tyoyo/lab/spell_correction


In [2]:
import dask.dataframe as dd
from smart_open import open
from pathlib import Path
import tarfile

In [3]:
JWTD_URL = 'http://nlp.ist.i.kyoto-u.ac.jp/DLcounter/lime.cgi?down=http://nlp.ist.i.kyoto-u.ac.jp/nl-resource/JWTD/jwtd.tar.gz&name=JWTD.tar.gz'

In [4]:
def download_data():
    if Path("data/jwtd").exists():
        return
    with open(JWTD_URL, "rb") as f:
        with tarfile.open(fileobj=f) as tar:
            tar.extractall("data")


In [5]:
download_data()

In [6]:
df = dd.read_json('data/jwtd/train.jsonl')

In [7]:
df.head()

,category,page,pre_rev,post_rev,pre_loss,post_loss,pre_text,post_text,diffs
0,substitution,11,8438207,8438306,107.91,94.64,仮名の「ん」で表される音は、後続の音によって などとにる。,仮名の「ん」で表される音は、後続の音によって などとなる。,"[{'pre': 'にる', 'post': 'なる'}]"
1,substitution,11,8340576,8347256,98.43,93.17,長母音と短母音を区別があり、母音の長さで意味が変わる。,長母音と短母音の区別があり、母音の長さで意味が変わる。,"[{'pre': 'を', 'post': 'の'}]"
2,substitution,11,10660707,10663485,89.56,85.26,これが正されるのは、江戸時代に本居宣長の登場してからのことである。,これが正されるのは、江戸時代に本居宣長が登場してからのことである。,"[{'pre': 'の', 'post': 'が'}]"
3,substitution,31,69177119,69190275,84.85,73.36,シャルべ（シャルべ・プラス・ヴァンドーム）,シャルベ（シャルベ・プラス・ヴァンドーム）,"[{'pre': 'シャルべ', 'post': 'シャルベ'}, {'pre': 'シャル..."
4,substitution,31,49861281,49874437,285.34,275.48,ジャンヌ・ダルクの活躍などもあり、1435年のアラスの和約でブルゴーニュ派と和解して勢力を伸...,ジャンヌ・ダルクの活躍などもあり、1435年のアラスの和約でブルゴーニュ派と和解して勢力を伸...,"[{'pre': '改めと', 'post': '改めて'}]"


In [8]:
len(df)

515328

In [9]:
train, dev = df.random_split([500000/515328, 1- 500000/515328], random_state=0)

In [17]:
len(train), len(dev)

(500071, 15257)

In [15]:
with open('gs://tyoyo/jwtd/v1.0/train.tsv', 'w') as f:
    train[['pre_text', 'post_text']].compute().to_csv(f, sep='\t', header=None, index=False)

with open('gs://tyoyo/jwtd/v1.0/dev.tsv', 'w') as f:
    dev[['pre_text', 'post_text']].compute().to_csv(f, sep='\t', header=None, index=False)

In [16]:
test = dd.read_json('data/jwtd/test.jsonl')
with open('gs://tyoyo/jwtd/v1.0/test.tsv', 'w') as f:
    test[['pre_text', 'post_text']].compute().to_csv(f, sep='\t', header=None, index=False)

In [18]:
readme = """
## JWTD preprocessed dataset

このデータセットは[日本語Wikipedia入力誤りデータセット](http://nlp.ist.i.kyoto-u.ac.jp/?日本語Wikipedia入力誤りデータセット)を機械翻訳で解くためにtsv形式で前処理をしたデータセットです。

## Data format

tsvファイルで 修正前の文 \t 修正後の文

```shell
$ head -3 train.tsv
仮名の「ん」で表される音は、後続の音によって などとにる。	仮名の「ん」で表される音は、後続の音によって などとなる。
長母音と短母音を区別があり、母音の長さで意味が変わる。	長母音と短母音の区別があり、母音の長さで意味が変わる。
これが正されるのは、江戸時代に本居宣長の登場してからのことである。	これが正されるのは、江戸時代に本居宣長が登場してからのことである。
```


## Stats

Datasize

* train: 500071
* dev: 15257
* test: 8542

(公開されているのはtrain/testで区切られているため、trainとdevの区切り方は任意。)

## Changelogs

### v1.0

init
"""


with open('gs://tyoyo/jwtd/README.md', 'w') as f:
    f.write(readme)